In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import openai
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()

In [4]:

query = "select userid from usr_applied_jobs_info where unixtimestamp>1707040266"

mycursor.execute(query)
result = mycursor.fetchall()

In [5]:
active_users = pd.DataFrame(result,columns=['userid'])

In [6]:
active_users.drop_duplicates(inplace=True)

In [7]:
active_users

,userid
0,953498
1,564565
2,2721267
3,310469
4,2038480
...,...
200082,473028
200088,2560115
200090,477492
200108,2698650


In [10]:
ulist = tuple(active_users['userid'].tolist())

In [11]:

query = f"select id, resume_last_updated, resume_first_uploaded from profile_info where id in {ulist}"

mycursor.execute(query)
result = mycursor.fetchall()

In [18]:
rdata = pd.DataFrame(result,columns=['id','last_upload','first_upload'])

In [19]:
rdata['start_date'] = pd.to_datetime(rdata['first_upload'])
rdata['end_date'] = pd.to_datetime(rdata['last_upload'])

# Calculate the difference in years
rdata['year_difference'] = rdata['end_date'] - rdata['start_date']
rdata['year_difference'] = rdata['year_difference'] / pd.Timedelta(days=365.25)

# To round the years to a sensible number of decimal places, e.g., 2 decimal places
rdata['year_difference'] = rdata['year_difference'].round(2)


In [26]:
select = rdata[rdata['year_difference']>3]

In [31]:
select

,id,last_upload,first_upload,start_date,end_date,year_difference
0,17,2024-01-19 12:32:39,2013-06-25 14:21:21,2013-06-25 14:21:21,2024-01-19 12:32:39,10.57
2,77,2024-01-05 12:05:06,2013-01-26 11:07:45,2013-01-26 11:07:45,2024-01-05 12:05:06,10.94
3,79,2024-01-21 16:58:45,2013-09-27 12:03:50,2013-09-27 12:03:50,2024-01-21 16:58:45,10.32
4,92,2023-12-25 21:28:18,2012-10-19 05:27:47,2012-10-19 05:27:47,2023-12-25 21:28:18,11.18
5,170,2023-12-01 18:59:46,2012-10-21 01:25:09,2012-10-21 01:25:09,2023-12-01 18:59:46,11.11
...,...,...,...,...,...,...
18846,1957200,2024-02-05 10:39:56,2021-01-31 16:12:10,2021-01-31 16:12:10,2024-02-05 10:39:56,3.01
18858,1958252,2024-02-06 07:54:29,2021-02-01 19:26:21,2021-02-01 19:26:21,2024-02-06 07:54:29,3.01
18862,1958558,2024-02-05 20:23:11,2021-02-02 00:21:01,2021-02-02 00:21:01,2024-02-05 20:23:11,3.01
18878,1960617,2024-02-08 11:48:19,2021-02-04 10:54:32,2021-02-04 10:54:32,2024-02-08 11:48:19,3.01


In [29]:
from user_info import get_user_info

In [32]:
r_text = {}
for uid in tqdm(select['id'].tolist()):
    response = get_user_info(uid)
    try:
        r_text[uid] = response['resume'][0]
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17003/17003 [02:04<00:00, 136.80it/s]


In [33]:
import json
with open('../custom_data/relevant_users_january24.json','w') as f:
    json.dump(r_text,f)